* eda
* prepare dataset for bert formet
    * add special tokens to the **start** and **end** of each sentence
    * pad and truncate
    * care about `mask`, `id`,... (1 for token, 0 for pad)
* finetuning
* ensemble
---
Know the difference among
* tokenizer
* special token
* attributes of token


* Exploring the pre-trained BERT model
* Extracting embeddings from pre-trained BERT
* Extracting embeddings from all encoder layers of BERT
* Fine-tuning BERT for downstream tasks

In [46]:
import torch
from transformers import *
# model = RobertaModel.from_pretrained('roberta-base')
# model.save_pretrained('./model')

In [47]:
model = RobertaModel.from_pretrained('./model')

loading configuration file ./model/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.32.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file ./model/pytorch_model.bin


All model checkpoint weights were used when initializing RobertaModel.

All the weights of RobertaModel were initialized from the model checkpoint at ./model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaModel for predictions without further training.


In [48]:
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# tokenizer.save_pretrained('./tokenizer')
tokenizer = RobertaTokenizer.from_pretrained('./tokenizer')

loading file vocab.json
loading file merges.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [95]:
sentence = "I love Paris"
tokens = tokenizer.tokenize(sentence)
print(tokens)
tokens = ['<s>'] + tokens + ['</s>']
print(tokens)
tokens = tokens + ["<pad>"] * 2
print(tokens)
attention_mask = [1 if i != "<pad>" else 0 for i in tokens ]
print(attention_mask)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

['I', 'Ġlove', 'ĠParis']
['<s>', 'I', 'Ġlove', 'ĠParis', '</s>']
['<s>', 'I', 'Ġlove', 'ĠParis', '</s>', '<pad>', '<pad>']
[1, 1, 1, 1, 1, 0, 0]
[0, 100, 657, 2201, 2, 1, 1]


In [96]:
# torch.tensor(token_ids).unsqueeze(dim = 0) 
# why do we need to unsqueeze at dim = 0 ?
# because we have 1 sentence, that's mean our batch_size is 1
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

In [97]:
outputs = model(token_ids, attention_mask = attention_mask)

In [98]:

outputs.last_hidden_state.shape


torch.Size([1, 7, 768])

In [99]:
hidden_rep, cls_head = outputs[0], outputs[1]
print(hidden_rep.shape)
print(cls_head.shape)
# [1,7,768] ~ [batch_size, sequence_length, hidden_size]

In [111]:
hidden_rep[0][0].shape 
# 768
# với mỗi batch, ta có 1 câu (sequence_length = số token + pad)
#  tensor có torch.Size([768]) là biểu diễn của 1 token


torch.Size([768])